In [1]:
!pip install pygame
!pip install numpy

In [ ]:
import sys  
import os        
import pygame
import numpy as np
import math
import time
import random
from itertools import cycle


pygame.init()
pygame.font.init()
BG_COLOR = (0, 0, 0)
screen = pygame.display.set_mode((1024, 768), pygame.RESIZABLE)
SCREEN_SIZE = screen.get_size()


class Game:
    def __init__(self, screen):
        self.screen = screen
        self.starship = Starship()
        self.bullets = []
        self.asteroids = []
        self.boosters = []
        self.boosters_timeouts = {"Rapid_fire": 0, "Shield": 0, "Triple_bullets": 0}
        self.booster_handlers = {"Rapid_fire": self.rapid_fire, "Shield": self.shield, "Triple_bullets": self.triple_bullets}
        self.mouse_pressed = False
        self.fire_rate = 0.2 # 1 / 5
        self.max_bullets = 1
        self.background_image = pygame.image.load(os.path.join("images", "background.png"))
        self.animation = self.load_animation(os.path.join("images", "animations"), repeat=3)
        self.score = 0
        self.score_font = pygame.font.SysFont('Comic Sans MS', 40)
        
    def load_animation(self, path, repeat=1):
        files = sorted(os.listdir(path))
        animation = []
        for file in files:
            for _ in range(repeat):
                animation.append(pygame.image.load(os.path.join(path, file)))
        return cycle(animation)
        
        
    def booster_manager(self, frame):
        boosters_rects = []
        for booster in self.boosters:
            boosters_rects.append(booster.rect)
            
        hit = self.starship.rect.collidelist(boosters_rects)
        if hit != -1:
            booster_type = self.boosters[hit].type
            if self.boosters_timeouts[booster_type] == 0:
                self.boosters_timeouts[booster_type] = time.time() + 10
                self.booster_handlers[booster_type]("activate")
            elif self.boosters_timeouts[booster_type] > 0:
                self.boosters_timeouts[booster_type] += 10
            del self.boosters[hit]
        
        for booster_type, timeout in self.boosters_timeouts.items():
            if time.time() > timeout > 0:
                self.boosters_timeouts[booster_type] = 0
                self.booster_handlers[booster_type]("deactivate")
        
        if frame % 400 == 0 and frame != 0:
            self.boosters.append(Booster())
        
        
    def handle_events(self, frame):
        for event in pygame.event.get():     
            if event.type == pygame.QUIT:  
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                self.mouse_pressed = True
            if event.type == pygame.MOUSEBUTTONUP:
                self.mouse_pressed = False
        if self.mouse_pressed == True and (time.time() - self.starship.last_bullet_time) > self.fire_rate:
            new_bullet = self.starship.fire(self.max_bullets)
            self.bullets += new_bullet
        if frame % 100 == 0:
            self.asteroids.append(Asteroid())
                
    def move_objects(self, objects_list):
        for obj_idx, _ in enumerate(objects_list):
            for o_idx, _ in enumerate(objects_list[obj_idx]):
                objects_list[obj_idx][o_idx].move()
                
    def draw(self, objects_list):
        # self.screen.fill(BG_COLOR)
        self.screen.blit(next(self.animation), (0, 0))
        for objects in objects_list:
            for obj in objects:
                self.screen.blit(obj.image, obj.rect)
        self.draw_texts()
        pygame.display.update()
        
    def draw_texts(self):
        text_image = self.score_font.render(f"Очки: {self.score}", True, (255, 255, 255))
        self.screen.blit(text_image, (SCREEN_SIZE[0] * 6/7, 0))
        
    def check_collisions(self):
        asteroids_rects = []
        for asteroid in self.asteroids:
            asteroids_rects.append(asteroid.rect)
            
        for idx, bullet in enumerate(self.bullets):
            if bullet.pos[0] > SCREEN_SIZE[0] or bullet.pos[1] > SCREEN_SIZE[1] or bullet.pos[0] < 0 or bullet.pos[1] < 0:
                del self.bullets[idx]
                
        for idx, bullet in enumerate(self.bullets):
            hit = bullet.rect.collidelist(asteroids_rects)   # -1 если столкновений нет
            if hit != -1:
                self.asteroids += self.asteroids[hit].explode()
                self.score += 1
                del self.asteroids[hit]
                del asteroids_rects[hit]
                del self.bullets[idx]
        
        
        hit = self.starship.rect.collidelist(asteroids_rects)
        if hit != -1:
            if self.boosters_timeouts["Shield"] > 0:
                del self.asteroids[hit]
            else:
                sys.exit()
                
    def rapid_fire(self, mode):    # "activate" "deactivate"
        if mode == "activate":
            self.fire_rate /= 2
        elif mode == "deactivate":
            self.fire_rate *= 2
            
    def shield(self, mode):    # "activate" "deactivate"
        if mode == "activate":
            self.starship.original_image = pygame.image.load(os.path.join("images", "Starship_with_shield.png"))
        elif mode == "deactivate":
            self.starship.original_image = pygame.image.load(os.path.join("images", "starship.png"))
            
    def triple_bullets(self, mode):    # "activate" "deactivate"
        if mode == "activate":
            self.max_bullets = 3
        elif mode == "deactivate":
            self.max_bullets = 1
    
                
    def run(self):
        frame = 0
        clock = pygame.time.Clock()
        while True:
            clock.tick(60)
            self.handle_events(frame)
            self.check_collisions()
            self.booster_manager(frame)
            self.move_objects([[self.starship], self.bullets, self.asteroids])
            self.draw([self.boosters, [self.starship], self.bullets, self.asteroids])
            frame += 1
        
class Starship:
    def __init__(self):
        self.pos = np.array([SCREEN_SIZE[0] / 2, SCREEN_SIZE[1] / 2])
        self.original_image = pygame.image.load(os.path.join("images", "starship.png"))
        self.image = self.original_image
        self.rect = starship_rect = self.image.get_rect(center=self.pos)
        self.last_bullet_time = 0
        
    def move(self):
        mouse_pos = pygame.mouse.get_pos()   # 300 200
        direction = mouse_pos - self.pos
        angle = self.calculate_angle(mouse_pos)
        self.speed = direction / 30
        self.pos += self.speed                # 250 250  + 4 5 = 254 255
        self.image = pygame.transform.rotate(self.original_image, int(angle))
        self.rect = self.image.get_rect(center=self.pos)
        
    def calculate_angle(self, mouse_pos):
        rel_x, rel_y = mouse_pos - self.pos    
        angle = (180 / math.pi) * -math.atan2(rel_y, rel_x) + 90
        return angle
    
    def fire(self, bullet_num):
        self.last_bullet_time = time.time()
        new_bullets = []
        for i in range(bullet_num):
            if bullet_num == 1:
                angle_offset = 0
            else:
                angle_offset = -15 + 30 / (bullet_num - 1) * i
            new_bullets.append(Bullet(self.pos.copy(), angle_offset))
        return new_bullets
    
    
class Bullet:
    original_image = pygame.image.load(os.path.join("images", "bullet.png"))
    def __init__(self, pos, angle_offset=0):
        self.pos = pos
        mouse_pos = pygame.mouse.get_pos()
        asr = math.pi / 180 * angle_offset
        self.direction = mouse_pos - self.pos
        self.direction[0] = self.direction[0] * math.cos(asr) - self.direction[1] * math.sin(asr)      
        self.direction[1] = self.direction[0] * math.sin(asr) + self.direction[1] * math.cos(asr)      
        self.speed = self.direction / max(abs(self.direction)) * 10 
        self.angle = self.calculate_angle(mouse_pos) - angle_offset
        self.image = pygame.transform.rotate(self.original_image, int(self.angle))
        self.rect = self.image.get_rect(center=self.pos)
        
    def move(self):
        self.pos += self.speed
        self.rect = self.image.get_rect(center=self.pos)
    
        
    def calculate_angle(self, mouse_pos):
        rel_x, rel_y = mouse_pos - self.pos    
        angle = (180 / math.pi) * -math.atan2(rel_y, rel_x) + 90
        return angle
        
class Asteroid:
    ast_variants = [("small", pygame.image.load(os.path.join("images", "ast1_small.png")), (10, 10)),
                   ("small", pygame.image.load(os.path.join("images", "ast2_small.png")), (10, 10)),
                   ("small", pygame.image.load(os.path.join("images", "ast3_small.png")), (10, 10)),
                   ("small", pygame.image.load(os.path.join("images", "ast4_small.png")), (10, 10)),
                   ("medium", pygame.image.load(os.path.join("images", "ast1_medium.png")), (40, 40)),
                   ("medium", pygame.image.load(os.path.join("images", "ast2_medium.png")), (40, 40)),
                   ("medium", pygame.image.load(os.path.join("images", "ast3_medium.png")), (40, 40)),
                   ("medium", pygame.image.load(os.path.join("images", "ast4_medium.png")), (40, 40)),
                   ("large", pygame.image.load(os.path.join("images", "ast1_large.png")), (105, 105)),
                   ("large", pygame.image.load(os.path.join("images", "ast2_large.png")), (105, 105)),
                   ("large", pygame.image.load(os.path.join("images", "ast3_large.png")), (105, 105)),
                   ("large", pygame.image.load(os.path.join("images", "ast4_large.png")), (105, 105))]

    def __init__(self, pos=None, speed=None, ast_type=None):
        if (pos is None) and (speed is None) and (ast_type is None):
            self.init_rand_asteroid()
        else:
            self.init_asteroid_fragment(pos, speed, ast_type)
        
        
    def init_rand_asteroid(self):
        self.type, self.original_image, hitbox_shape = random.choice(self.ast_variants)
        self.image = self.original_image
        self.w, self.h = hitbox_shape
        self.pos = random.choice([self.left_pos, self.right_pos, self.top_pos, self.bottom_pos])()
        self.rect = self.image.get_rect(center=self.pos)
        self.direction = pygame.mouse.get_pos() - self.pos
        self.speed = self.direction / 300
        
    def init_asteroid_fragment(self, pos, speed, ast_type):
        ast_type_variants = list(filter(lambda x: x[0] == ast_type, self.ast_variants))
        self.type, self.original_image, hitbox_shape = random.choice(ast_type_variants)
        self.pos = pos
        self.image = self.original_image
        self.rect = self.image.get_rect(center=self.pos)
        self.speed = speed
        
        
    def move(self):
        self.check_borders()
        self.pos += self.speed
        self.rect = self.image.get_rect(center=self.pos)
        
    def left_pos(self):
        return np.array([-150, random.uniform(0, SCREEN_SIZE[1])])
    
    def right_pos(self):
        return np.array([SCREEN_SIZE[0] + 150, random.uniform(0, SCREEN_SIZE[1])])
    
    def top_pos(self):
        return np.array([random.uniform(0, SCREEN_SIZE[0]), -150])
    
    def bottom_pos(self):
        return np.array([random.uniform(0, SCREEN_SIZE[0]), SCREEN_SIZE[1] + 150])
    
    def check_borders(self):
        if self.pos[0] > (SCREEN_SIZE[0] + 150):
            self.pos[0] = -150
        elif (self.pos[0] + 150) < 0:
            self.pos[0] = SCREEN_SIZE[0] + 150
        if (self.pos[1] - 150) > SCREEN_SIZE[1]:
            self.pos[1] = -150
        elif (self.pos[1] + 150) < 0:
            self.pos[1] = SCREEN_SIZE[1] + 150
        # self.rect = self.image.get_rect(center=self.pos)
        
    def explode(self):
        fragments = []
        if self.type == "large":
            fragments += [Asteroid(self.pos_offset(), self.speed_offset(), "medium") for _ in range(2)]
        if self.type == "medium":
            fragments += [Asteroid(self.pos_offset(), self.speed_offset(), "small") for _ in range(3)]
        if self.type == "small":
            pass
        return fragments
    
    def speed_offset(self):
        offset = self.speed * 0.5 + np.random.uniform(-0.5, 0.5, 2)
        return offset
    
    def pos_offset(self):
        offset = self.pos + np.random.uniform(-10, 10, 2)
        return offset
    

class Booster:
    booster_types = {"Rapid_fire": pygame.image.load(os.path.join("images", "Rapid_fire.png")),
                     "Shield": pygame.image.load(os.path.join("images", "Shield.png")),
                     "Triple_bullets": pygame.image.load(os.path.join("images", "Triple_bullets.png"))}
    def __init__(self):
        self.type = random.choice(list(self.booster_types.keys()))
        self.image = self.booster_types[self.type]
        self.pos = np.array([random.uniform(100, SCREEN_SIZE[0] - 100), random.uniform(100, SCREEN_SIZE[1] - 100)])
        self.rect = self.image.get_rect(center=self.pos)
        
        
game = Game(screen)
game.run()

In [1]:
import os

In [7]:
sorted(os.listdir("images/animations"))

['frame_00_delay-0.1s.png',
 'frame_01_delay-0.1s.png',
 'frame_02_delay-0.1s.png',
 'frame_03_delay-0.1s.png',
 'frame_04_delay-0.1s.png',
 'frame_05_delay-0.1s.png',
 'frame_06_delay-0.1s.png',
 'frame_07_delay-0.1s.png',
 'frame_08_delay-0.1s.png',
 'frame_09_delay-0.1s.png',
 'frame_10_delay-0.1s.png',
 'frame_11_delay-0.1s.png',
 'frame_12_delay-0.1s.png',
 'frame_13_delay-0.1s.png',
 'frame_14_delay-0.1s.png',
 'frame_15_delay-0.1s.png',
 'frame_16_delay-0.1s.png',
 'frame_17_delay-0.1s.png',
 'frame_18_delay-0.1s.png',
 'frame_19_delay-0.1s.png']

In [4]:
import pygame
pygame.font.init()
a = pygame.font.SysFont('Comic Sans MS', 40)

In [5]:
text_image = self.score_font.render(f"Очки: {self.score}", True, (255, 255, 255))




self.screen.blit(text_image, (SCREEN_SIZE[0] * 6/7, 0))

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'bold',
 'get_ascent',
 'get_bold',
 'get_descent',
 'get_height',
 'get_italic',
 'get_linesize',
 'get_underline',
 'italic',
 'metrics',
 'render',
 'set_bold',
 'set_italic',
 'set_underline',
 'size',
 'underline']

In [12]:
b = 5

In [9]:
print("У Васи " + str(b) + " яблок")

У Васи 6 яблок


In [13]:
print(f"У Васи {b} яблок")

У Васи 5 яблок


In [16]:
с = f"У Васи {b} {b + 1} яблок"

In [17]:
с

'У Васи 5 6 яблок'